# Full outcome model

In [1]:
# import required modules
import numpy as np
import pandas as pd
from math import sqrt
from scipy import stats

from classes.pathway import SSNAP_Pathway
from classes.clinical_outcome import Clinical_outcome

In [2]:
# TEMPORARY 
import warnings
warnings.filterwarnings('ignore')

In [3]:
hospital_performance = pd.read_csv('data/hospital_performance.csv')

In [4]:
mrs_dists = pd.read_csv('data/mrs_dist_probs_cumsum.csv', index_col='Stroke type')

In [5]:
mrs_dists

,0,1,2,3,4,5,6
Stroke type,,,,,,,
pre_stroke_nlvo,0.582881,0.745419,0.848859,0.951082,0.993055,1.000000,1.0
pre_stroke_nlvo_ivt_deaths,0.576469,0.737219,0.839522,0.940620,0.982131,0.989000,1.0
pre_stroke_lvo,0.417894,0.560853,0.679283,0.843494,0.957269,1.000000,1.0
pre_stroke_lvo_ivt_deaths,0.403644,0.541728,0.656119,0.814731,0.924626,0.965900,1.0
pre_stroke_lvo_mt_deaths,0.402850,0.540662,0.654829,0.813128,0.922807,0.964000,1.0
no_treatment_nlvo,0.197144,0.460000,0.580032,0.707768,0.855677,0.917702,1.0
no_effect_nlvo_ivt_deaths,0.197271,0.460000,0.577583,0.702252,0.845244,0.904454,1.0
t0_treatment_nlvo_ivt,0.429808,0.630000,0.738212,0.848427,0.929188,0.956300,1.0
no_treatment_lvo,0.050000,0.129000,0.265000,0.429000,0.676000,0.811000,1.0


In [6]:

# Set general model parameters
scenario_counter = 0
trials = 100

# Set up dataframes.

# Record these measures...
outcome_results_columns = [
    'Thrombolysis_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
]

# ... with these stats...
results_types = [
    '_(median)',
    '_(low_5%)',
    '_(high_95%)',
    '_(mean)',
    '_(stdev)',
    '_(95ci)',
]
# ... and gather all combinations of measure and stat here:
results_columns = []
for column in outcome_results_columns:
    columns_here = [column + ending for ending in results_types]
    results_columns += columns_here
# Also store onset to needle time:
results_columns += ['Onset_to_needle_(mean)']


results_df = pd.DataFrame(columns=results_columns)

# trial dataframe is set up each scenario, but define column names here
trial_columns = [
    'Thrombolysis_rate_(%)',
    'LVO_IVT_mean_shift',
    'LVO_MT_mean_shift',
    'nLVO_IVT_mean_shift',
    'onset_to_needle'
    ]

# Iterate through hospitals
for hospital in hospital_performance.iterrows():
    scenario_counter += 1
    print(f'Scenario {scenario_counter}', end='\r')

    # Get data for one hospital
    hospital_name = hospital[0]
    hospital_data = hospital[1]
    run_data = hospital_data
    calibration = 1.0

    # Set up trial results dataframe
    trial_df = pd.DataFrame(columns=trial_columns)

    patients_per_run = int(run_data['admissions'])

    for trial in range(trials):

        patient_array = SSNAP_Pathway(hospital_name, hospital_data)
        patient_array.run_trial()

        number_of_patients = len(patient_array.patient_array_onset_to_needle_mins.ravel())
        
        # Set everyone to nLVO:
        dummy_stroke_type_code = np.full(number_of_patients, 1)
        
        # Pass each pair of times into the outcome model separately.
        # for t, time_to_ivt in len(patient_array.patient_array_onset_to_needle_mins):
        # Initiate the object:
        clinical_outcome = Clinical_outcome(mrs_dists, number_of_patients)
        # Import patient array data:
        clinical_outcome.each_patient_time_to_ivt_mins.data = \
            patient_array.patient_array_onset_to_needle_mins
        clinical_outcome.each_patient_received_ivt_bool.data = \
            patient_array.patient_array_thrombolysis_conditions_met_bool
        clinical_outcome.each_patient_stroke_type_code.data = \
            dummy_stroke_type_code
        
        # Calculate outcomes:
        results, combo_results = clinical_outcome.calculate_outcomes()
        
        mask = (
            patient_array.patient_array_thrombolysis_conditions_met_bool == 1)
        onset_to_needle_mins_masked = \
                patient_array.patient_array_onset_to_needle_mins[mask].mean()


        # Save scenario results to dataframe
        result = [
            patient_array.patient_array_thrombolysis_rate_percent,
            results['lvo_ivt_mean_valid_patients_mean_mrs_shift'],
            results['lvo_mt_mean_valid_patients_mean_mrs_shift'],
            results['nlvo_ivt_mean_valid_patients_mean_mrs_shift'],
            onset_to_needle_mins_masked
        ]
        # print('result', result)
        trial_df.loc[trial] = result
        


    trial_result = []
    
    for column in outcome_results_columns:
        results_here = [
            trial_df[column].median(),
            trial_df[column].quantile(0.05),
            trial_df[column].quantile(0.95),
            trial_df[column].mean(),
            trial_df[column].std(),
            (trial_df[column].mean() -
                stats.norm.interval(0.95, loc=trial_df[column].mean(),
                scale=trial_df[column].std() / sqrt(trials))[0]),
        ]
        trial_result += results_here
    trial_result += [trial_df['onset_to_needle'].mean()]
    
    # add scenario results to results dataframe
    results_df.loc[hospital_name] = trial_result

# Apply calibration
results_df['calibration'] = calibration
for col in list(results_df):
    if 'Percent_Thrombolysis' in col or 'Additional_good_outcomes' in col:
        results_df[col] *= calibration

# round all results to 2 decimal places and return
# results_df = results_df.round(2)
# return (results_df)


Print some info about the final trial:

In [7]:
print(clinical_outcome)

There are 597 patients and the base mRS distributions are: 
pre_stroke_nlvo array([0.582881, 0.745419, 0.848859, 0.951082, 0.993055, 1.      ,
       1.      ])
pre_stroke_nlvo_ivt_deaths array([0.57646931, 0.73721939, 0.83952155, 0.9406201 , 0.9821314 ,
       0.989     , 1.        ])
pre_stroke_lvo array([0.417894, 0.560853, 0.679283, 0.843494, 0.957269, 1.      ,
       1.      ])
pre_stroke_lvo_ivt_deaths array([0.40364381, 0.54172791, 0.65611945, 0.81473085, 0.92462613,
       0.9659    , 1.        ])
pre_stroke_lvo_mt_deaths array([0.40284982, 0.54066229, 0.65482881, 0.81312822, 0.92280732,
       0.964     , 1.        ])
no_treatment_nlvo array([0.19714427, 0.46      , 0.58003201, 0.70776826, 0.85567719,
       0.91770198, 1.        ])
no_effect_nlvo_ivt_deaths array([0.19727076, 0.46      , 0.57758322, 0.70225237, 0.84524353,
       0.90445421, 1.        ])
t0_treatment_nlvo_ivt array([0.42980772, 0.63      , 0.73821237, 0.84842726, 0.92918765,
       0.95630045, 1.        ])
n

In [8]:
results_df

,Thrombolysis_rate_(%)_(median),Thrombolysis_rate_(%)_(low_5%),Thrombolysis_rate_(%)_(high_95%),Thrombolysis_rate_(%)_(mean),Thrombolysis_rate_(%)_(stdev),Thrombolysis_rate_(%)_(95ci),LVO_IVT_mean_shift_(median),LVO_IVT_mean_shift_(low_5%),LVO_IVT_mean_shift_(high_95%),LVO_IVT_mean_shift_(mean),...,LVO_MT_mean_shift_(stdev),LVO_MT_mean_shift_(95ci),nLVO_IVT_mean_shift_(median),nLVO_IVT_mean_shift_(low_5%),nLVO_IVT_mean_shift_(high_95%),nLVO_IVT_mean_shift_(mean),nLVO_IVT_mean_shift_(stdev),nLVO_IVT_mean_shift_(95ci),Onset_to_needle_(mean),calibration
0,15.201192,13.107303,17.444113,15.187779,1.278352,0.250552,NaN,NaN,NaN,NaN,...,NaN,NaN,0.010043,0.008470,0.011514,0.010040,0.000925,0.000181,162.646638,1.0
1,14.960630,13.381452,17.064742,15.073491,1.077798,0.211245,NaN,NaN,NaN,NaN,...,NaN,NaN,0.010327,0.009239,0.011871,0.010417,0.000780,0.000153,153.161010,1.0
2,8.000000,6.200000,9.800000,7.866000,1.029683,0.201814,NaN,NaN,NaN,NaN,...,NaN,NaN,0.004611,0.003713,0.005819,0.004651,0.000668,0.000131,184.600784,1.0
3,10.022779,7.972665,12.323462,10.225513,1.366702,0.267869,NaN,NaN,NaN,NaN,...,NaN,NaN,0.006185,0.004719,0.007532,0.006172,0.000884,0.000173,180.732075,1.0
4,9.090909,6.163636,11.654545,9.000000,1.771202,0.347149,NaN,NaN,NaN,NaN,...,NaN,NaN,0.005109,0.003503,0.006677,0.005168,0.001050,0.000206,189.827841,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
127,11.200000,8.360000,14.400000,11.120000,1.942195,0.380663,NaN,NaN,NaN,NaN,...,NaN,NaN,0.005902,0.004169,0.007967,0.006066,0.001177,0.000231,198.960663,1.0
128,11.173184,8.086592,13.407821,10.972067,1.651382,0.323665,NaN,NaN,NaN,NaN,...,NaN,NaN,0.006738,0.004887,0.008349,0.006709,0.001034,0.000203,178.211326,1.0
129,7.795100,6.002227,10.244989,8.011136,1.252882,0.245560,NaN,NaN,NaN,NaN,...,NaN,NaN,0.005111,0.003908,0.006604,0.005164,0.000818,0.000160,167.723533,1.0
130,10.804020,9.415829,12.688442,10.879397,1.035956,0.203044,NaN,NaN,NaN,NaN,...,NaN,NaN,0.006862,0.005758,0.008034,0.006923,0.000684,0.000134,170.387591,1.0


In [9]:
results_df.to_csv('full_outcome_pathway_results_2.csv')